In [6]:
import sqlite3
import pandas as pd

# Step 1: Connect to the database
conn = sqlite3.connect("Chinook_Sqlite.sqlite")




**View All Available Tables**


In [5]:
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables


,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


**View data inside each relevant table used in SQL analysis**

In [7]:
# Show top 5 rows from Invoice table
invoice_df = pd.read_sql("SELECT * FROM Invoice LIMIT 5;", conn)
print("\nInvoice Table:")
display(invoice_df)

# Show top 5 rows from InvoiceLine table
invoiceline_df = pd.read_sql("SELECT * FROM InvoiceLine LIMIT 5;", conn)
print("\nInvoiceLine Table:")
display(invoiceline_df)

# Show top 5 rows from Customer table
customer_df = pd.read_sql("SELECT * FROM Customer LIMIT 5;", conn)
print("\nCustomer Table:")
display(customer_df)

# Show top 5 rows from Track table
track_df = pd.read_sql("SELECT * FROM Track LIMIT 5;", conn)
print("\nTrack Table:")
display(track_df)

# Show top 5 rows from Album table (if used)
album_df = pd.read_sql("SELECT * FROM Album LIMIT 5;", conn)
print("\nAlbum Table:")
display(album_df)


Invoice Table:


,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86



InvoiceLine Table:


,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1



Customer Table:


,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4



Track Table:


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99



Album Table:


,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


**SQL Queries for Business Questions**
1. Top-Selling Products (Tracks)

In [8]:
query = """
SELECT
    t.Name AS Track_Name,
    SUM(il.Quantity) AS Total_Quantity_Sold
FROM InvoiceLine il
JOIN Track t ON il.TrackId = t.TrackId
GROUP BY t.Name
ORDER BY Total_Quantity_Sold DESC
LIMIT 10;
"""
top_selling = pd.read_sql(query, conn)
top_selling


,Track_Name,Total_Quantity_Sold
0,The Trooper,5
1,Untitled,4
2,The Number Of The Beast,4
3,Sure Know Something,4
4,Hallowed Be Thy Name,4
5,Eruption,4
6,Where Eagles Dare,3
7,Welcome Home (Sanitarium),3
8,Sweetest Thing,3
9,Surrender,3


2. Revenue per Region (Country)

In [9]:
query = """
SELECT
    c.Country,
    ROUND(SUM(i.Total), 2) AS Total_Revenue
FROM Invoice i
JOIN Customer c ON i.CustomerId = c.CustomerId
GROUP BY c.Country
ORDER BY Total_Revenue DESC;
"""
revenue_by_country = pd.read_sql(query, conn)
revenue_by_country


,Country,Total_Revenue
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48
5,United Kingdom,112.86
6,Czech Republic,90.24
7,Portugal,77.24
8,India,75.26
9,Chile,46.62


3. Monthly Sales Performance

In [31]:
import pandas as pd
from IPython.display import display

# Assuming 'conn' is your database connection (e.g., SQLite)
query = """
SELECT
    strftime('%Y-%m', i.InvoiceDate) AS Invoice_Month,
    ROUND(SUM(i.Total), 2) AS Monthly_Revenue
FROM Invoice i
GROUP BY Invoice_Month
ORDER BY Invoice_Month;
"""

# Load the query result into a pandas DataFrame
monthly_sales = pd.read_sql(query, conn)

# Display the DataFrame with headers
display(monthly_sales)

,Invoice_Month,Monthly_Revenue
0,2009-01,35.64
1,2009-02,37.62
2,2009-03,37.62
3,2009-04,37.62
4,2009-05,37.62
5,2009-06,37.62
6,2009-07,37.62
7,2009-08,37.62
8,2009-09,37.62
9,2009-10,37.62


 4. Use a Window Function — Rank Countries by Revenue

In [12]:
query = """
SELECT
    Country,
    Total_Revenue,
    RANK() OVER (ORDER BY Total_Revenue DESC) AS Revenue_Rank
FROM (
    SELECT
        c.Country,
        SUM(i.Total) AS Total_Revenue
    FROM Invoice i
    JOIN Customer c ON i.CustomerId = c.CustomerId
    GROUP BY c.Country
);
"""
country_rank = pd.read_sql(query, conn)
country_rank


,Country,Total_Revenue,Revenue_Rank
0,USA,523.06,1
1,Canada,303.96,2
2,France,195.10,3
3,Brazil,190.10,4
4,Germany,156.48,5
5,United Kingdom,112.86,6
6,Czech Republic,90.24,7
7,Portugal,77.24,8
8,India,75.26,9
9,Chile,46.62,10
